In [1]:
import src.data_processing as data

In [2]:
code_map_json = "./src/data_processing/code_map.json"
code_map = data.code.load_code_map_json(code_map_json)

In [54]:
code_data = "./data/code-only/hmda_2017_ca_all-records_code_only_no_county.csv"

df = data.numeric.read_raw_csv_numeric(code_data)

In [7]:
df.columns

Index(['as_of_year', 'respondent_id', 'agency_abbr', 'agency_code',
       'loan_type', 'property_type', 'loan_purpose', 'owner_occupancy',
       'loan_amount_000s', 'preapproval', 'action_taken', 'msamd',
       'state_abbr', 'state_code', 'county_name', 'county_code',
       'census_tract_number', 'applicant_ethnicity', 'co_applicant_ethnicity',
       'applicant_race_1', 'applicant_race_2', 'applicant_race_3',
       'applicant_race_4', 'applicant_race_5', 'co_applicant_race_1',
       'co_applicant_race_2', 'co_applicant_race_3', 'co_applicant_race_4',
       'co_applicant_race_5', 'applicant_sex', 'co_applicant_sex',
       'applicant_income_000s', 'purchaser_type', 'denial_reason_1',
       'denial_reason_2', 'denial_reason_3', 'rate_spread', 'hoepa_status',
       'lien_status', 'edit_status', 'sequence_number', 'population',
       'minority_population', 'hud_median_family_income',
       'tract_to_msamd_income', 'number_of_owner_occupied_units',
       'number_of_1_to_4_famil

In [55]:
for i in range(1,6):
    df['applicant_race_'+str(i)] = df['applicant_race_'+str(i)].str[:1]
    df['co_applicant_race_'+str(i)] = df['co_applicant_race_'+str(i)].str[:1]
    print(df['applicant_race_'+str(i)].value_counts())

5    969817
6    271228
2    212429
7    160432
3     68421
4     16313
1     15819
Name: applicant_race_1, dtype: Int64
5    8954
4    1591
3    1230
2    1046
1     411
Name: applicant_race_2, dtype: Int64
5    547
3    230
4     82
1     37
2     24
Name: applicant_race_3, dtype: Int64
4    173
5     60
2      9
3      6
1      2
Name: applicant_race_4, dtype: Int64
5    167
2      6
1      5
3      2
4      1
Name: applicant_race_5, dtype: Int64


In [56]:
df.to_csv("./data/code-only/hmda_2017_ca_all-records_code_only_modify_sex.csv")

In [8]:
df['action_taken'].value_counts()

1    877753
4    241369
6    233447
3    226258
5     81948
2     53535
7        80
8        69
Name: action_taken, dtype: Int64

In [10]:
df['preapproval'].value_counts()

3    1506675
2     179061
1      28723
Name: preapproval, dtype: Int64

In [11]:
df['applicant_race_2'].value_counts()

5.0    8954
4.0    1591
3.0    1230
2.0    1046
1.0     411
Name: applicant_race_2, dtype: Int64

In [13]:
len(code_map['county'])

58

## Drop unused columns from code-only

In [57]:
drop_columns = ['respondent_id', 'census_tract_number', 'sequence_number', 'application_date_indicator', # Identity
                'as_of_year', 'state_abbr', 'state_code',  # Common Info
                'county_name',  # Overlapping feature
                'applicant_ethnicity', 'co_applicant_ethnicity', # Overlap with race info
                'agency_abbr', 'agency_code', # Abandoned features
                'msamd', 'rate_spread', 'hoepa_status',
               'lien_status','population',
               'minority_population', 'hud_median_family_income',
               'tract_to_msamd_income', 'number_of_owner_occupied_units',
               'number_of_1_to_4_family_units', # Overlapping geolocational info
                 'edit_status', 'purchaser_type'] # Unused info
#                'loan_type', 'property_type', 'loan_purpose', 'owner_occupancy',
#                'loan_amount_000s', 'preapproval', 'action_taken', 
#                'county_code',
#                'applicant_race_1', 'applicant_race_2', 'applicant_race_3',
#                'applicant_race_4', 'applicant_race_5', 'co_applicant_race_1',
#                'co_applicant_race_2', 'co_applicant_race_3', 'co_applicant_race_4',
#                'co_applicant_race_5', 'applicant_sex', 'co_applicant_sex',
#                'applicant_income_000s', , 'denial_reason_1',
#                'denial_reason_2', 'denial_reason_3']
dropped = df.drop(drop_columns, axis=1)

In [58]:
df['applicant_race_2'].value_counts()

5    8954
4    1591
3    1230
2    1046
1     411
Name: applicant_race_2, dtype: Int64

In [59]:
dropped.to_csv('./data/cleaned/hmda_2017_ca_all-records_drop_column.csv')

In [60]:
dropped['applicant_race_2'].value_counts()

5    8954
4    1591
3    1230
2    1046
1     411
Name: applicant_race_2, dtype: Int64

In [22]:
dropped['applicant_sex'].value_counts()

1    996254
2    427287
4    160050
3    130868
Name: applicant_sex, dtype: Int64

## Compacted Features

In [4]:
dropped = data.numeric.read_raw_csv_numeric('./data/cleaned/hmda_2017_ca_all-records_drop_column.csv')

In [5]:
dropped['has_co_applicant'] = dropped['co_applicant_sex'] != '5'
dropped['approved'] = dropped['action_taken'].isin(['1','2','4','6','8'])
for i in range(1,6):
    dropped['applicant_race_'+str(i)].fillna('',inplace=True)
dropped['applicant_races'] = dropped[['applicant_race_1','applicant_race_2','applicant_race_3',
                                'applicant_race_4','applicant_race_5']].agg(''.join, axis=1)

In [6]:
dropped['applicant_races'].value_counts()

5        968697
6        271228
2        207459
7        160432
3         66919
          ...  
241           1
15432         1
523           1
132           1
514           1
Name: applicant_races, Length: 87, dtype: int64

In [7]:
import pandas as pd

In [25]:
dropped = dropped.drop('Unnamed: 0', axis=1)
dropped.columns

Index(['loan_type', 'property_type', 'loan_purpose', 'owner_occupancy',
       'loan_amount_000s', 'preapproval', 'action_taken', 'county_code',
       'applicant_race_1', 'co_applicant_race_1', 'co_applicant_race_2',
       'co_applicant_race_3', 'co_applicant_race_4', 'co_applicant_race_5',
       'applicant_sex', 'co_applicant_sex', 'applicant_income_000s',
       'denial_reason_1', 'denial_reason_2', 'denial_reason_3',
       'has_co_applicant', 'approved', 'applicant_races'],
      dtype='object')

In [29]:
dropped['approved'] = dropped['action_taken'].isin(['1','2','4','6','8'])

In [31]:
dropped.to_csv('./data/cleaned/hmda_2017_ca_all-records_agg_sex.csv',index=False)

In [33]:
new_dropped = data.numeric.read_raw_csv_numeric('./data/cleaned/hmda_2017_ca_all-records_agg_sex.csv',numeric_code=True)
new_dropped.dtypes # Need to fix the applicant_races

loan_type                float64
property_type            float64
loan_purpose             float64
owner_occupancy          float64
loan_amount_000s         float64
preapproval              float64
action_taken             float64
county_code              float64
applicant_race_1         float64
co_applicant_race_1      float64
co_applicant_race_2      float64
co_applicant_race_3      float64
co_applicant_race_4      float64
co_applicant_race_5      float64
applicant_sex            float64
co_applicant_sex         float64
applicant_income_000s    float64
denial_reason_1          float64
denial_reason_2          float64
denial_reason_3          float64
has_co_applicant            bool
approved                    bool
applicant_races            int64
dtype: object

In [164]:
features = [# 'loan_type', 'property_type', 'loan_purpose', 'owner_occupancy',
       'loan_amount_000s', 'owner_occupancy','loan_purpose', # 'preapproval', 'action_taken', 'county_code',
       # 'applicant_race_1', 'co_applicant_race_1', 'co_applicant_race_2',
       # 'co_applicant_race_3', 'co_applicant_race_4', 'co_applicant_race_5', 'co_applicant_sex', 
       'applicant_sex','applicant_income_000s',
       # 'denial_reason_1', 'denial_reason_2', 'denial_reason_3', 'approved',
       'has_co_applicant', 'applicant_races']
feat_df = dropped[features]
print(feat_df.columns)

Index(['loan_amount_000s', 'owner_occupancy', 'loan_purpose', 'applicant_sex',
       'applicant_income_000s', 'has_co_applicant', 'applicant_races'],
      dtype='object')


In [165]:
feat_df.applicant_races.astype('string', copy=False)

0          5
1          2
2          6
3          5
4          5
          ..
1714454    7
1714455    7
1714456    7
1714457    7
1714458    7
Name: applicant_races, Length: 1714459, dtype: string

In [166]:
def numeric_feature(s, log=False):
    import numpy as np
    if log:
        s = np.log2(s)
    print(s)
    return s

def multi_enumerate_feature(s):
    import collections
    data = s.str.split(pat="")
    print(s.name)
    data = data.apply(collections.Counter)
    multi = pd.DataFrame.from_records(data).fillna(value=0)
    multi = multi.drop('',axis=1)
    multi = multi.rename(columns=dict(zip(multi.columns, [s.name+'_'+x for x in multi.columns])))
    return multi

def enumerate_feature(s):
    return pd.get_dummies(s, prefix=s.name)

In [167]:
approved = dropped.approved.astype('int')

In [168]:
# features = pd.DataFrame(index=feat_df.index)
features = []
for name in feat_df.columns:
    if feat_df[name].dtype == 'float':
        print("Numerical feature:", name)
        f = numeric_feature(feat_df[name],log=True)
        print(f.shape, f.dtypes)
        features.append(f)
    elif name == 'applicant_races':
        print("Multi-value enumeration feature:", name)
        f = multi_enumerate_feature(feat_df[name])
        print(f.shape, f.dtypes)
        features.append(f)
    else:
        print("Enumeration feature:", name)
        f = enumerate_feature(feat_df[name])
        print(f.shape, f.dtypes)
        features.append(f)
features = pd.concat(features + [approved], axis=1)
print(features.columns)
print(features.dtypes)

Numerical feature: loan_amount_000s
0           9.154818
1           7.531381
2          10.075479
3           8.703904
4           8.566054
             ...    
1714454     6.906891
1714455     6.643856
1714456     7.033423
1714457     7.169925
1714458     9.060696
Name: loan_amount_000s, Length: 1714459, dtype: float64
(1714459,) float64
Enumeration feature: owner_occupancy
(1714459, 3) owner_occupancy_1    uint8
owner_occupancy_2    uint8
owner_occupancy_3    uint8
dtype: object
Enumeration feature: loan_purpose
(1714459, 3) loan_purpose_1    uint8
loan_purpose_2    uint8
loan_purpose_3    uint8
dtype: object
Enumeration feature: applicant_sex
(1714459, 4) applicant_sex_1    uint8
applicant_sex_2    uint8
applicant_sex_3    uint8
applicant_sex_4    uint8
dtype: object
Numerical feature: applicant_income_000s
0          7.169925
1          5.672425
2          8.118941
3          6.965784
4          6.426265
             ...   
1714454         NaN
1714455         NaN
1714456         N

In [169]:
features.columns

Index(['loan_amount_000s', 'owner_occupancy_1', 'owner_occupancy_2',
       'owner_occupancy_3', 'loan_purpose_1', 'loan_purpose_2',
       'loan_purpose_3', 'applicant_sex_1', 'applicant_sex_2',
       'applicant_sex_3', 'applicant_sex_4', 'applicant_income_000s',
       'has_co_applicant_False', 'has_co_applicant_True', 'applicant_races_5',
       'applicant_races_2', 'applicant_races_6', 'applicant_races_3',
       'applicant_races_1', 'applicant_races_7', 'applicant_races_4',
       'approved'],
      dtype='object')

In [170]:
features.shape

(1714459, 22)

In [171]:
print(features.isna().sum())
features.dropna(inplace=True)

loan_amount_000s             945
owner_occupancy_1              0
owner_occupancy_2              0
owner_occupancy_3              0
loan_purpose_1                 0
loan_purpose_2                 0
loan_purpose_3                 0
applicant_sex_1                0
applicant_sex_2                0
applicant_sex_3                0
applicant_sex_4                0
applicant_income_000s     195886
has_co_applicant_False         0
has_co_applicant_True          0
applicant_races_5              0
applicant_races_2              0
applicant_races_6              0
applicant_races_3              0
applicant_races_1              0
applicant_races_7              0
applicant_races_4              0
approved                       0
dtype: int64


In [172]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import LeaveOneOut, KFold, train_test_split
# pca = PCA(n_components=3)

### Drop NaN values

In [ ]:
y = features.approved
x = features.drop(['approved'],axis=1)
x_std = StandardScaler().fit_transform(x)
print(x.shape, y.shape, x_std.shape)

In [176]:
# pca = PCA(n_components=3)
# pca.fit(f_std)
# div = pca_2.transform(features)

NameError: name 'pca_2' is not defined

In [194]:
def train_lr(x, y):
    LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.01, random_state=42)

    hit_rate = []
    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    print(sum(y_train)/len(y_train), sum(y_test)/len(y_test))
    clf = LR.fit(X_train, y_train)
    # clf = LR.fit(f_std,approved)
    pred = clf.predict(X_test)
    print(pred.shape)
    
    rate_approved = len([1 for t in y_test if t == 1])/len(y_test)
    rate_disapproved = len([1 for t in y_test if t == 0])/len(y_test)
    hit_samples = [p for p, q in zip(pred, y_test) if p == q]
    hit_rate = len(hit_samples)/len(y_test)
    print(rate_approved, rate_disapproved, hit_rate)
    print(sum(hit_samples)/len(hit_samples))

In [185]:
# kf = KFold(n_splits=10)
train_lr(x_std, y)

(1502460, 21) (15177, 21) (1502460,) (15177,)
0.80986781678048 0.8082625024708441
(15177,)
0.8082625024708441 0.19173749752915595 0.8083942808196614
0.999266443882957


Clearly, we see there's nearly no difference between all-approved and the model

### Data Augmentation

In [190]:
disapproved = features.loc[features.approved == 0]

In [199]:
import copy
aug = copy.deepcopy(features)
for i in range(1,4):
    print("Starting Data Augmentation for the {0}-th time".format(i))
    aug = pd.concat([aug, disapproved], axis=0)
    aug_y = aug.approved
    aug_x = aug.drop(['approved'],axis=1)
    aug_x_std = StandardScaler().fit_transform(aug_x)
    print(aug_y.shape, aug_x.shape, aug_x_std.shape)
    train_lr(aug_x_std, aug_y)

Starting Data Augmentation for the 1-th time
(1806213,) (1806213, 21) (1806213, 21)
(1788150, 21) (18063, 21) (1788150,) (18063,)
0.6804910102620026 0.6776836627359796
(18063,)
0.6776836627359796 0.32231633726402037 0.685711122183469
0.9533344098175359
Starting Data Augmentation for the 2-th time
(2094789,) (2094789, 21) (2094789, 21)
(2073841, 21) (20948, 21) (2073841,) (20948,)
0.5867166287097226 0.5873591751002483
(20948,)
0.5873591751002483 0.4126408248997518 0.6181497040290242
0.741910572244961


In [201]:
print("Starting Data Augmentation for the 3-th time")
aug = pd.concat([aug, disapproved], axis=0)
aug_y = aug.approved
aug_x = aug.drop(['approved'],axis=1)
aug_x_std = StandardScaler().fit_transform(aug_x)
print(aug_y.shape, aug_x.shape, aug_x_std.shape)
train_lr(aug_x_std, aug_y)

Starting Data Augmentation for the 4-th time
(2383365,) (2383365, 21) (2383365, 21)
(2359531, 21) (23834, 21) (2359531,) (23834,)
0.5156651893956893 0.5174540572291684
(23834,)
0.5174540572291684 0.48254594277083157 0.605269782663422
0.4896021073062526
